# Cap tube rise

In the previous notebook we dealt with some scaling issues preventing dynamic visualisation of our capillary rise case, this was partially fixed, we now write the dynamic visualisation so that we can debug the subsequent test case models faster.




Test case based on the experiment by Lunowa et al. (2022) _Dynamic Effects during the Capillary Rise of Fluids in Cylindrical Tubes_.

In [14]:
import numpy as np
import math
import copy
import os
import sys

# Add base directory (~/ddg/ddgclib/) to sys.path from notebook location
#module_path = os.path.abspath(os.path.join('../..'))
#if module_path not in sys.path:
#    sys.path.append(module_path)

from scipy.spatial import Delaunay

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# ddg imports (from ddgclib package)
from ddgclib import *
from ddgclib._complex import Complex
from ddgclib._curvatures import *
from ddgclib._capillary_rise_flow import *
from ddgclib._capillary_rise import *
from ddgclib._eos import *
from ddgclib._misc import *
from ddgclib._plotting import *

# barycentric imports (from ddgclib.barycentric subpackage)
from ddgclib.barycentric._duals import compute_vd, triang_dual, plot_dual, plot_dual_mesh_2D, _set_boundary

# compute duals (and Delaunay dual from a set of points)
from ddgclib.barycentric._duals import compute_vd, _merge_local_duals_vector, triang_dual, plot_dual

# Plots
from ddgclib.barycentric._duals import  plot_dual_mesh_2D, plot_dual_mesh_3D

# Geometry and dual computations
from ddgclib.barycentric._duals import area_of_polygon, e_star, volume_of_geometric_object, plot_dual, v_star

# Boundary geometry
from ddgclib.barycentric._duals import  _set_boundary, _find_plane_equation, _find_intersection, _reflect_vertex_over_edge

# Area computations
from ddgclib.barycentric._duals import d_area

# Volume computations (including helper functions)
from ddgclib.barycentric._duals import _signed_volume_parallelepiped, _volume_parallelepiped
# DDG gradient operations on primary edges (for continuum)
from ddgclib.barycentric._duals import dP, du, dudt

From the paper we expect to reach equilibrium after ∼0.05 s when using the parameters defined in the next cell:

In [15]:
r_list = np.array([0.375, 0.5, 0.65]) * 1e-3  # m, Tube radii for 3 experiments
#r = 0.5#e-3  # mm (not m!), Default tube radius
r = r_list[1]  # mm (not m!), Default tube radius

gamma = 0.0728  # N/m, surface tension of water at 20 deg C
mu = 0.001  # Pa s, viscosity
rho = 997  # kg/m3, density

g = 9.81  # m/s2
# Parameters from EoS:
T_0 = 273.15 + 25  # K, initial tmeperature
P_0 = 101.325  # kPa, ambient pressure
#gamma = IAPWS(T_0)  # N/m, surface tension of water at 20 deg C
theta_p = 9.99* np.pi/180.0  # Three-phase contact angle (degrees immediately converted to radius

# Initial condition
theta_i = 0#2*np.pi  #TODO: Doesn't really do anything yet unless equilibrium is set to True

# Jurin equilibrium height in m:
h_jurin = 2 * gamma * np.cos(theta_p) / (rho * g * r)
r*1e3, h_jurin *1e3

(0.5, 29.321894204747586)

NOTE: From the paper we expect a rise time is roughly 0.3 s

The initial condition is atmospheric pressure at the bottom of the tube (assume the tube is "instantly" inserted and appears below the water line)

The rest of the pressure initial condition should follow rho g h increase.

Different boundary conditions will be explored.

In [16]:
# Import the relevant functions
from Dynamic_caprise_3D_tube import run_cap_rise_simulation, _cap_rise_meniscus_init, cap_rise_init_dyn, cube_to_tube

nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.001875
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.000625
verts = [[ 1.53080850e-20 -2.50000000e-04  0.00000000e+00]
 [-1.17851130e-04 -2.84517797e-04  0.00000000e+00]
 [-1.45524772e-04 -2.70524772e-04 -1.56250000e-04]
 [-5.71364241e-05 -1.82136424e-04 -4.68750000e-04]
 [ 5.71364241e-05 -1.82136424e-04 -4.68750000e-04]
 [ 1.45524772e-04 -2.70524772e-04 -1.56250000e-04]
 [ 1.17851130e-04 -2.84517797e-04  0.00000000e+00]
 [-1.76776695e-04 -1.76776695e-04  0.00000000e+00]
 [-2.70524772e-04 -1.45524772e-04 -

KeyboardInterrupt: 

In [5]:
r, r * 0.5e1

(0.0005, 0.0025)

In [6]:
ps.remove_all_structures()

In [18]:
# Initiation
%matplotlib notebook
import polyscope as ps
if 0:
    r = 0.5

height = r * 0.5e1

#height = 0.5
HC = cube_to_tube(r, refinements=1, height=height)

#HC.plot_complex()

# Compute the initial duals
compute_vd(HC, cdist=1e-8)

# Plot the initial complex
if 1:
    #vi = HC.V[(0.0, 0.0, 0.0)]
    #vi = HC.V[(0.5, 0.0, 0.0)]  # Vertex No. 17
    vi = HC.V[(0.5e-3, 0.0, 0.0)]  # Vertex No. 17
    #vi = HC.V[(0.3535533905932738, 0.35355339059327373, 0.5)]
    print(f'vi.x = {vi.x}')
    #plot_dual(vi, HC, length_scale=1e3, point_radii=1e-5)
    length_scale = 1e-3
    plot_dual(vi, HC, length_scale=length_scale,
                      point_radii=1e-5)
    ps.look_at((0., -10.*length_scale, 0.), (0., 0., 0.))  # Side
    ps.look_at((0., -10.*length_scale, 2.*length_scale), (0., 0., 0.))  # Side raise
    #ps.show()
    #ps.remove_surface_mesh("Dual face")
    ps.remove_all_structures()


nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.001875
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.000625
vi.x = (0.0005, 0.0, 0.0)
[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 580.82.09
verts = [[ 2.50000000e-04  0.00000000e+00  0.00000000e+00]
 [ 2.84517797e-04  1.17851130e-04  0.00000000e+00]
 [ 2.70524772e-04  1.45524772e-04 -1.56250000e-04]
 [ 1.82136424e-04  5.71364241e-05 -4.68750000e-04]
 [ 1.82136424e-04 -5.71364241e-05 -4.68750000e-04]
 [ 2.70524772e-04 -1.45524772e-04 -1.56250000e-04]
 [ 2.84517797e-04 -1.1785113

In [19]:
# Move some vertices:
if 1: # Turn off for testing
    vi = HC.V[(0.0, 0.0, 0.0)]
    #vi = HC.V[(0.3535533905932738, 0.35355339059327373, 0.5)]
    vt_new = (0.0, 0.0, (0.5 + 2e-1)*1e-3)
    v_new = HC.V.move(vi, vt_new)
    vi = v_new

    # Postprocessing
    HC.V.merge_all(1e-12)
    # Clear the dual cache
    for v in HC.V:
        v.vd = set()

    HcVd = copy.copy(HC.Vd)
    for vd in HcVd:
        HC.Vd.cache.pop(vd.x)

    # Recompute the dual:
    compute_vd(HC, cdist=1e-7)

    # Plot the final complex
    if 1:
        plot_dual(vi, HC, length_scale=length_scale,
                      point_radii=1e-5)
        ps.look_at((0., -10.*length_scale, 0.), (0., 0., 0.))  # Side
        ps.look_at((0., -10.*length_scale, 2.*length_scale), (0., 0., 0.))  #

        #ps.show()
        #ps.remove_surface_mesh("Dual face")
        ps.remove_all_structures()


verts = [[-1.76776695e-04 -1.76776695e-04  3.50000000e-04]
 [-2.84517797e-04 -1.17851130e-04  2.33333333e-04]
 [-2.70524772e-04 -1.45524772e-04  1.87500000e-05]
 [-1.45524772e-04 -2.70524772e-04  1.87500000e-05]
 [-1.17851130e-04 -2.84517797e-04  2.33333333e-04]
 [ 1.53080850e-20 -2.50000000e-04  3.50000000e-04]
 [ 1.17851130e-04 -2.84517797e-04  2.33333333e-04]
 [ 1.45524772e-04 -2.70524772e-04  1.87500000e-05]
 [ 5.71364241e-05 -1.82136424e-04 -2.93750000e-04]
 [-5.71364241e-05 -1.82136424e-04 -2.93750000e-04]
 [-1.14272848e-04 -1.14272848e-04  3.75000000e-05]
 [-1.82136424e-04 -5.71364241e-05 -2.93750000e-04]
 [ 1.53080850e-20  2.50000000e-04  3.50000000e-04]
 [-1.17851130e-04  2.84517797e-04  2.33333333e-04]
 [-1.45524772e-04  2.70524772e-04  1.87500000e-05]
 [-5.71364241e-05  1.82136424e-04 -2.93750000e-04]
 [ 5.71364241e-05  1.82136424e-04 -2.93750000e-04]
 [ 1.45524772e-04  2.70524772e-04  1.87500000e-05]
 [ 1.17851130e-04  2.84517797e-04  2.33333333e-04]
 [ 2.50000000e-04  0.00

# Energy minimisation approach
## Sparse


In [1]:
run_cap_rise_simulation(refinements=1, r=r, height=height, h_jurin=h_jurin)

NameError: name 'cube_to_tube' is not defined

In [10]:
1e-3*(h_jurin - vt_new[2])

2.9494072995810799374e-05

# Medium refinement

In [11]:
run_cap_rise_simulation(refinements=2, r=r, height=height, h_jurin=h_jurin)

nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = 0.0
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.00125
nv[2] = 0.0
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.0

KeyboardInterrupt: 

# Fine

In [ ]:
run_cap_rise_simulation(refinements=3, r=r, height=height, h_jurin=h_jurin)

nv[2] = -0.0025
nv[2] = 0.0
nv[2] = -0.00125
nv[2] = -0.001875
nv[2] = -0.000625
nv[2] = -0.0015625
nv[2] = -0.0021875
nv[2] = -0.0021875
nv[2] = -0.0009375
nv[2] = -0.0009375
nv[2] = -0.0003125
nv[2] = 0.0
nv[2] = -0.0009375
nv[2] = -0.0025
nv[2] = -0.0021875
nv[2] = -0.001875
nv[2] = -0.0015625
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = -0.0003125
nv[2] = 0.0
nv[2] = -0.0009375
nv[2] = -0.001875
nv[2] = -0.0015625
nv[2] = -0.00125
nv[2] = -0.0025
nv[2] = -0.0021875
nv[2] = -0.000625
nv[2] = -0.0003125
nv[2] = 0.0
nv[2] = -0.0003125
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.0015625
nv[2] = 0.0
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = -0.0025
nv[2] = -0.0021875
nv[2] = -0.0021875
nv[2] = -0.001875
nv[2] = -0.0015625
nv[2] = -0.0025
nv[2] = -0.001875
nv[2] = -0.00125
nv[2] = -0.000625
nv[2] = 0.0
nv[2] = -0.0021875
nv[2] = -0.0015625
nv[2] = -0.0015625
nv[2] = -0.0003125
nv[2] = -0.0009375
nv[2] = -0.0

# Conclusion:

This appears to work as execpted now, but it is unforutnately very slow. cleaning breaking out of the while loop also doesn't seem to work without crashing.